In [1]:
import pandas as pd
import sys
import numpy as np
import re

### 讀取實價登錄原始檔

In [2]:
df = pd.read_excel("/Users/chouhouren/Desktop/論文資料_新/python code/實價登錄整理/實價登錄列表合併.xlsx")

### 取出本研究所需之資料欄位

In [3]:
df = df[['編號', '屋齡', '主要用途_x', '主要建材_x',
         '鄉鎮市區', '交易標的',
         '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態',
         '建物移轉總面積(平方公尺)', '建物現況格局-房', '建物現況格局-廳','建物現況格局-衛',
         '總價(元)', '單價(元/平方公尺)', 
         '車位類別','車位移轉總面積(平方公尺)', '車位總價(元)',
         '有無備註欄(Y/N)', '備註','交易標的橫坐標', '交易標的縱坐標']]

### 1.交易標的篩選  (將僅有土地車位建物欄位剃除)

In [4]:
df = df[(df['交易標的'] =="房地(土地+建物)") | (df['交易標的'] =="房地(土地+建物)+車位")] 

### 2.交易年月日   (將小於102年1月1日的資料剔除)

In [5]:
df['交易年月日'] = df['交易年月日'].astype('str').astype(int)

df = df[df['交易年月日'] >= 1020101]

### 3.建物型態篩選  (將非公寓、華廈、住宅大樓之欄位剔除)

In [6]:
df = df[(df['建物型態']  =="公寓(5樓含以下無電梯)")|(df['建物型態']  =="住宅大樓(11層含以上有電梯)")|(df['建物型態']  =="華廈(10層含以下有電梯)")]  

### 4.面積篩選  (將建物移轉面積小於3坪(9.9174平方公尺)剔除)

In [7]:
df = df[~(df['建物移轉總面積(平方公尺)']<9.9174)]     

### 5.屋齡篩選  (將屋齡小於1及空白剔除)

In [8]:
df = df[ (df['屋齡'] >1) & (df['屋齡'] != pd.isnull) ]

### 6.移轉層次篩選  (剃除2樓以下及非單一樓層交易)

In [9]:
df['移轉層次'] = df['移轉層次'].astype(str)
df['移轉層次'] =  df['移轉層次'].str.replace('\夾層', '夾')#

In [10]:
df  = df[~(df["移轉層次"].str.contains("一層"))
        &~(df["移轉層次"].str.contains("地下"))
        &~(df["移轉層次"].str.count("層") >=2)
        &~(df["移轉層次"]  == "nan")
        ] 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86776 entries, 3 to 209400
Data columns (total 24 columns):
編號               86776 non-null object
屋齡               86776 non-null float64
主要用途_x           86392 non-null object
主要建材_x           86765 non-null object
鄉鎮市區             86776 non-null object
交易標的             86776 non-null object
交易年月日            86776 non-null int64
交易筆棟數            86776 non-null object
移轉層次             86776 non-null object
總樓層數             86768 non-null object
建物型態             86776 non-null object
建物移轉總面積(平方公尺)    86776 non-null float64
建物現況格局-房         86776 non-null int64
建物現況格局-廳         86776 non-null int64
建物現況格局-衛         86776 non-null int64
總價(元)            86776 non-null int64
單價(元/平方公尺)       86776 non-null float64
車位類別             42983 non-null object
車位移轉總面積(平方公尺)    86776 non-null float64
車位總價(元)          86776 non-null int64
有無備註欄(Y/N)       86776 non-null object
備註               34513 non-null object
交易標的橫坐標          86776 non-null in

### 7.房間數篩選  (將房間數大於6間的資料剔除)

In [11]:
df = df[~(df['建物現況格局-房']>6)] 

### 8.主要用途篩選  (將非住商及住家用途之欄位剔除)

In [12]:
df = df[(df['主要用途_x']  == "住家用") |(df['主要用途_x']  == "住商用")]   

### 9.車位資料篩選(將交易標的中有車位但沒有車位面積之欄位剔除)

In [13]:
df = df[(df['車位移轉總面積(平方公尺)']>0)|(df['交易標的'] =="房地(土地+建物)")]          

#### 總價重新計算(總價元扣除車位總價)

In [14]:
#df["新總價"]=[]

for row in df["車位總價(元)"]:
    if row>0:
        df["新總價"] = df["總價(元)"]-df["車位總價(元)"]

#### 建物移轉總面記重新計算(建物總移轉面積扣除車位總面積)

In [15]:
#df["新面積"]=[]

for row in df["車位總價(元)"]:
    if row>0:
        df["新面積"] = df['建物移轉總面積(平方公尺)']-df['車位移轉總面積(平方公尺)']

### 10.特殊交易篩選(將備註欄有文字之資料剃除)

In [16]:
df['備註'] = df['備註'].fillna(0)

df = df[(df['備註'] == 0)] 

#### RESET df INDEX

In [17]:
df = df.reset_index(drop=True)

## 資料處理與需擬變數轉換

#### 總樓層數中文轉阿拉伯數字

In [18]:
dic ={ '零' : 0 , '一' : 1 , '二' : 2 , '三' : 3 , '四' : 4 , '五' : 5 , '六' : 6 , '七' : 7 , '八' : 8 , '九' : 9 , '十' : 10 , '百' : 100 , '千' : 1000 ,'萬' : 10000,
       '０' : 0 , '１' : 1 , '２' : 2 , '３' : 3 , '４' : 4 , '５' : 5 , '６' : 6 ,  '７' : 7 , '８' : 8 , '９' : 9 ,
                '壹' : 1 , '貳' : 2 , '叁' : 3 , '肆' : 4 , '伍' : 5 , '陸' : 6 , '柒' : 7 ,  '捌' : 8 , '玖' : 9 , '拾' : 10 , '佰' : 100 , '仟' : 1000 , '萬' : 10000 ,
       '億' : 100000000 }
def  getResultForDigit (a, encoding= "utf-8" ) :

    count = 0
    result = 0
    tmp = 0
    Billion = 0
    while count < len(a):
        tmpChr = a[count]
        #print tmpChr
        tmpNum = dic.get(tmpChr, None )
        #如果等於十或者百，千
        if tmpNum >= 10 :
            if tmp == 0 :
                tmp = 1
            result = result + tmpNum * tmp
            tmp = 0
        #如果是個位數
        elif tmpNum is  not  None :
            tmp = tmp * 10 + tmpNum
        count += 1
    result = result + tmp
    result = result + Billion
    return result

In [19]:
df['總樓層數'] = df['總樓層數'].astype(str)
df["總樓層數"]=df['總樓層數'].str.replace('層','')
df = df[(df['總樓層數']  != 'nan')]

In [20]:
b=[]

for row in df["總樓層數"]:
    c=getResultForDigit(row)
    b.append(c)
    
df["總樓層數"]=b
df  = df[~(df["總樓層數"]==0)]

#### 移轉層次中文轉阿拉伯數字

df["移轉層次"] =  df['移轉層次'].str.replace('，','')#1
df["移轉層次"] =  df['移轉層次'].str.replace('層','')#2
df["移轉層次"] =  df['移轉層次'].str.replace('機械房','')#3
df["移轉層次"] =  df['移轉層次'].str.replace('夾','')#4
df["移轉層次"] =  df['移轉層次'].str.replace('見其他登記事項','')#5
df["移轉層次"] =  df['移轉層次'].str.replace('走廊','')#6
df["移轉層次"] =  df['移轉層次'].str.replace('陽台','')#7
df["移轉層次"] =  df['移轉層次'].str.replace('露台','')#8
df["移轉層次"] =  df['移轉層次'].str.replace('電梯樓梯間','')#9
df["移轉層次"] =  df['移轉層次'].str.replace('屋頂突出物','')#10
df["移轉層次"] =  df['移轉層次'].str.replace('騎樓','')#11
df["移轉層次"] =  df['移轉層次'].str.replace('平台','')#12
df['移轉層次'] =  df['移轉層次'].str.replace('通道', '')#13
df['移轉層次'] =  df['移轉層次'].str.replace('見使用執照', '')#14
df['移轉層次'] =  df['移轉層次'].str.replace('門廳', '')#15
df['移轉層次'] =  df['移轉層次'].str.replace('防空避難室', '')#16
df['移轉層次'] =  df['移轉層次'].str.replace('防空避難室見使用執照', '')#17
df['移轉層次'] =  df['移轉層次'].str.replace('防空避難室停車場見使用執照', '')#18
df['移轉層次'] =  df['移轉層次'].str.replace('停車場', '')#19
df = df[(df['移轉層次']  != pd.isnull)]





In [21]:
unwanted_part = ['，', '層', '機械房', '夾', '見其他登記事項','走廊','陽台','露台','電梯樓梯間',
                 '屋頂突出物','騎樓','平台','通道','見使用執照','門廳','防空避難室','防空避難室見使用執照',
                 '防空避難室停車場見使用執照','停車場']

pat = '|'.join(['({})'.format(re.escape(c)) for c in unwanted_part])

df['移轉層次'] =  df['移轉層次'].str.replace(pat, '')
df = df[(df['移轉層次'] != pd.isnull)]
len(unwanted_part)

19

In [22]:
b=[]

for row in df["移轉層次"]:
    c=getResultForDigit(row)
    b.append(c)
    
df["移轉層次"]=b
df  = df[~(df["移轉層次"]==0)]

#### 建物類型轉虛擬變數

In [23]:
dummy_建物類型 = pd.get_dummies(pd.Series(list(df['建物型態'])))

#### 建物材料轉虛擬變數

In [24]:
建築材料 = [
              (df['主要建材_x'] == "加強磚造"),
              (df['主要建材_x'] == "磚造"),
              (df['主要建材_x'] == "見使用執照"),
              (df['主要建材_x'] =="見其他登記事項"),
              (df['主要建材_x'] =="壁式預鑄鋼筋混凝土造"),
              (df['主要建材_x'] =="鋼筋混凝土造"),
              (df['主要建材_x'] =="鋼骨混凝土造"),
              (df['主要建材_x'] =="鋼骨鋼筋混凝土造")
                                                 ]
          
              
choices = ['RB',"RB","RC","RC","RC","RC","SRC/SC","SRC/SC"]
df['建築材料歸類'] = np.select(建築材料, choices, default=0)

In [25]:
dummy_建物結構 = pd.get_dummies(pd.Series(list(df['建築材料歸類'])))

#### 車位數與類型整理

In [26]:
df['交易車位數'] = df['交易筆棟數'].apply(lambda x: x.split('車位')[-1])

In [27]:
#車位數
交易車位數 = [
              (df['車位總價(元)']  >0),
              (df['車位總價(元)'] == 0) ]
              
                            
choices = [0,df['交易車位數']]
df['新交易車位數'] = np.select(交易車位數, choices, default=0)

In [28]:
#車位類別轉換
車位類別 = [
              (df['車位總價(元)']  >0),
              (df['車位總價(元)'] == 0)]
              
                                                                                 
choices = ["無車位",df['車位類別']]
df['車位類別扣除車位總價'] = np.select(車位類別, choices)
df['車位類別扣除車位總價']=df['車位類別扣除車位總價'].fillna('無車位')

In [29]:
車位類別 = [
              (df['車位類別扣除車位總價'] == "一樓平面"),
              (df['車位類別扣除車位總價'] == "升降平面"),
              (df['車位類別扣除車位總價'] == "升降機械"),
              (df['車位類別扣除車位總價'] =="其他"),
              (df['車位類別扣除車位總價'] =="坡道平面"),
              (df['車位類別扣除車位總價'] =="坡道機械"),
              (df['車位類別扣除車位總價'] =="塔式車位")
                                                    ]
                            
choices = ['平面車位',"平面車位","機械車位","其他","平面車位","機械車位","機械車位"]
df['新車位類別'] = np.select(車位類別, choices, default="無車位")

In [30]:
dummy_車位類型 = pd.get_dummies(pd.Series(list(df['新車位類別'])))

#### 行政區轉虛擬變數

In [31]:
dummy_鄉鎮市區 = pd.get_dummies(pd.Series(list(df['鄉鎮市區'])))

#### 交易年度轉虛擬變數

In [32]:
交易日期 = [
              (df['交易年月日'] >1060630),
              (df['交易年月日'] >1060101),
              (df['交易年月日'] >1050630),
              (df['交易年月日'] >1050101),
              (df['交易年月日'] >1040630),
              (df['交易年月日'] >1040101),
              (df['交易年月日'] >1030630),
              (df['交易年月日'] >1030101),
              (df['交易年月日'] >1020630),
              (df['交易年月日'] >1020101),
                                                 ]
          
              
choices = ['106下',"106上","105下","105上","104下","104上","103下","103上","102下","102上"]
df['交易日期'] = np.select(交易日期, choices, default="102上")

In [33]:
dummy_交易日期 = pd.get_dummies(pd.Series(list(df['交易日期'])))

### 合併虛擬變數

In [34]:
df = pd.concat([df,dummy_建物結構,dummy_車位類型,dummy_鄉鎮市區,dummy_交易日期,dummy_建物類型],axis=1)

#### 將建物總面積轉成坪後平方取對數，新總價取對數

In [ ]:
df['新總價'] = np.log(df['新總價'])
df['新面積'] = np.log((df['新面積']*0.3025)**2)

In [35]:
df.columns

Index(['編號', '屋齡', '主要用途_x', '主要建材_x', '鄉鎮市區', '交易標的', '交易年月日', '交易筆棟數',
       '移轉層次', '總樓層數', '建物型態', '建物移轉總面積(平方公尺)', '建物現況格局-房', '建物現況格局-廳',
       '建物現況格局-衛', '總價(元)', '單價(元/平方公尺)', '車位類別', '車位移轉總面積(平方公尺)', '車位總價(元)',
       '有無備註欄(Y/N)', '備註', '交易標的橫坐標', '交易標的縱坐標', '新總價', '新面積', '建築材料歸類',
       '交易車位數', '新交易車位數', '車位類別扣除車位總價', '新車位類別', '交易日期', '0', 'RB', 'RC',
       'SRC/SC', '其他', '平面車位', '機械車位', '無車位', '中山區', '中正區', '信義區', '內湖區',
       '北投區', '南港區', '士林區', '大同區', '大安區', '文山區', '松山區', '萬華區', '102上', '102下',
       '103上', '103下', '104上', '104下', '105上', '105下', '106上', '106下',
       '住宅大樓(11層含以上有電梯)', '公寓(5樓含以下無電梯)', '華廈(10層含以下有電梯)'],
      dtype='object')

#### 取出實價登錄變數

In [36]:
df=df[['編號','新總價','新面積',
       '屋齡','移轉層次','總樓層數',
       '建物現況格局-房', '建物現況格局-廳','建物現況格局-衛',
       '102上', '102下','103上', '103下', '104上', '104下', '105上', '105下', '106上', '106下', 
       '公寓(5樓含以下無電梯)','華廈(10層含以下有電梯)', '住宅大樓(11層含以上有電梯)', 
       'RB', 'RC', 'SRC/SC', 
       '新交易車位數', '平面車位', '機械車位','其他', 
       '士林區', '大同區','大安區', '中山區', '中正區', '內湖區', '文山區', '北投區', '松山區', '南港區', '萬華區', '信義區',
       '有無備註欄(Y/N)','交易標的橫坐標', '交易標的縱坐標'

       
       ]]

In [37]:
df = df.drop_duplicates(subset=["編號"], keep="first")

In [38]:
df = df.dropna()

In [39]:
df.to_excel("實價登錄清理沒有處理總價.xlsx")